# **ETH Price and Users Behavior on Compound**

- The main purpose of this notebok is to first present a summary of statistics on and see if there are sEit correlations between e-market price of ETH and borrowers behaviours
- **Another idea to pursue is the assumption that people may take short and long positions depending on the ETH price trend (increase-bullish or decease-bearish)**  
- consider the amount of cETH-cUSDC and cUSDC-cETH pairs.
- Plot whether the Ethereum price trend (up or down) may influence people to take more long or short strategies

Importing the necessary modules

In [1]:
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import altair as alt
from datetime import datetime
import statsmodels.api as sm
from statsmodels.tsa.stattools import grangercausalitytests
from sklearn.preprocessing import StandardScaler
import seaborn as sns
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

Preparing the data & creating a df for ETH

In [12]:
csv_file_path = r'../data/daily_balance_sheet.csv'
df = pd.read_csv(csv_file_path)
liability = pd.read_csv(r'../data/daily_liability_matrix.csv')

col_drop = ['repayUSD','bufferUSD', 'haircutUSD', 'seizeUSD']
df.drop(col_drop, axis = 1)

csv_file_path2 = r'../data/daily_markets.csv'
markets = pd.read_csv(csv_file_path2)
markets['date'] = pd.to_datetime(markets['date'])
markets['utilization_ratio'] = markets['totalBorrows'] / ( markets['totalBorrows'] + markets['cash'] - markets['reserves'] )
blocks_data = pd.read_csv(r'../data/blocks_daily.csv')
liability['gas_fee_usd'] = blocks_data['gas_fee_usd'].pct_change() 
df['date'] = pd.to_datetime(df['date'])
df_eth = df[df['symbol'] == 'cETH']


Plotting total borrow, total collateral and total assets

In [ ]:
chart1 = alt.Chart(df[df['symbol'] == 'cUSDC']).mark_line().encode(
    x='date:T',
    y='totalBorrowUSD:Q',
    color=alt.value('blue'),
    tooltip=['date', 'totalBorrowUSD']
).properties(
    width=600,
    height=300
)
chart2 = alt.Chart(df[df['symbol'] == 'cUSDC']).mark_line().encode(
    x='date:T',
    y = 'totalCollateralUSD:Q',
    tooltip=['date:T', 'totalCollateralUSD:Q'],
    color=alt.value('orange')
).properties(
    width=600,
    height=300
)
chart3 = alt.Chart(df[df['symbol'] == 'cUSDC']).mark_line().encode(
    x='date:T',
    y = 'totalAssetsUSD:Q',
    tooltip=['date:T', 'totalAssetsUSD:Q'],
    color=alt.value('red')
).properties(
    width=600,
    height=300
)

(chart1 +chart2+chart3)#.resolve_scale(y='independent')



Plotting total assets for most used coins 

In [ ]:
#see https://messari.io/report/state-of-compound-q4-2021 for most used coin
most_used_coins = ['cETH', 'cUSDC', 'cBTC', 'cDAI', 'cUSDT']

df_copy = df
df_copy = df_copy[df_copy['symbol'].isin(most_used_coins)]

chart1 = alt.Chart(df_copy).mark_area(
    interpolate='linear',
    line=True
).encode(
    x='date:T',
    y=alt.Y('totalAssetsUSD:Q',stack = None, title='Total Assets USD'),
    color='symbol:N',
    tooltip=['date:T', 'totalAssetsUSD:Q', 'symbol:N']
).properties(
    title='Total Assets USD Over Time',
    width=600,
    height=300
)


chart1


Total borrow and total assets for ETH

In [ ]:

chart1 = alt.Chart(df_eth).mark_line().encode(
    x = 'date:T',
    y = 'totalBorrowUSD:Q',
    tooltip=['date:T', 'totalLiabilitiesUSD:Q'],
    color=alt.value('blue')
)

chart2 = alt.Chart(df_eth).mark_line().encode(
    x = 'date:T',
    y = 'totalAssetsUSD:Q',
    tooltip=['date:T', 'totalCollateralUSD:Q'],
    color=alt.value('orange')
)

(chart1 + chart2).resolve_scale(y='independent') | (chart1+chart2).properties(title = 'Total borrow and assets in cETH, assets in orange')



Total assets and liabilities in USDC

In [ ]:
chart1 = alt.Chart(df[df['symbol'] == 'cUSDC']).mark_area().encode(
    x='date:T',
    y=alt.Y('totalAssetsUSD:Q'),
    color=alt.Color('variable:N', scale=alt.Scale(scheme='category10')),
    tooltip=['date:T', 'value:Q'],
).transform_fold(
    ['tokensUSD', 'reservesUSD', 'interAssetsUSD'],
    as_=['variable', 'value']
).properties(
    title='Total assets in USDC',
    width=600,
    height=400
)

chart2 = alt.Chart(df[df['symbol'] == 'cUSDC']).mark_area().encode(
    x='date:T',
    y=alt.Y('totalLiabilitiesUSD:Q'),
    color=alt.Color('variable:N', scale=alt.Scale(scheme='category10')),
    tooltip=['date:T', 'value:Q'],
).transform_fold(
    ['bufferUSD', 'depositsUSD', 'interLiabilitiesUSD', 'netWorthUSD'],
    as_=['variable', 'value']
).properties(
    title='Total liabilities in USDC',
    width=400,
    height=400
)
chart1|chart2


Reserves of coins over time

In [ ]:
most_used_coins = ['cETH', 'cUSDC', 'cBTC', 'cDAI', 'cUSDT']

chart = alt.Chart(df_copy).mark_line().encode(
    x='date:T',
    y='reservesUSD:Q',
    color='symbol:N',
    tooltip=['date:T', 'reservesUSD:Q', 'symbol:N']
).properties(
    title='Reserves for All Coins Over Time',
    width=600,
    height=300
)
chart




Plotting ETH price and checking correlation of col USDC- bor ETH and vice-versa
- First I have to create a dataset for ETH market (because 'underlyingPriceUSD' is not in 'Liability' but in 'daily_markets'
- Then I filter, first I only choose ETH as collateral in 'liability' then for the same dataset I filter for only borrowing USDC

In [3]:
#Creating df for ETH
ETH_markets = markets.loc[markets['symbol'] == 'cETH']
ETH_USD = liability[liability['symbol_col'] == 'cETH']
ETH_USD = ETH_USD[ETH_USD['symbol_bor'] == 'cUSDC']

ETH_USD['date'] = pd.to_datetime(ETH_USD['date']) #to merge with ETH_markets 
ETH_USD = pd.merge(ETH_USD, ETH_markets[['date', 'underlyingPriceUSD', 'borrowRate', 'supplyRate', 'totalBorrows', 'totalSupply', 'utilization_ratio']], on='date')
ETH_USD['matchBorrow'] = ETH_USD['matchBorrowUSD']/ETH_USD['underlyingPriceUSD']
chart1 = alt.Chart(ETH_USD).mark_line().encode(
    x = 'date:T',
    y = 'underlyingPriceUSD:Q',
    tooltip=['date:T', 'underlyingPriceUSD:Q'],
    color=alt.value('orange')
).properties(
    width=600,
    height=300
).interactive()

chart2 = alt.Chart(ETH_USD).mark_line().encode(
    x = 'date:T',
    y = 'matchBorrow:Q',
    tooltip=['date:T', 'matchBorrow:Q'],
    color=alt.value('black')
).properties(
    width=600,
    height=300
).interactive()
(chart1 +chart2).properties(title = "Price of ETH (orange), and quantity of USDC borrowed with ETH(black)").resolve_scale(y='independent')


"""We do same but this time, use USD to borrow ETH"""
USD_ETH = liability[liability['symbol_col'] == 'cUSDC']
USD_ETH = USD_ETH[USD_ETH['symbol_bor'] == 'cETH']
USD_ETH['date'] = pd.to_datetime(USD_ETH['date']) #to merge with ETH_markets 
USD_ETH= pd.merge(USD_ETH, ETH_markets[['date', 'underlyingPriceUSD', 'borrowRate', 'supplyRate', 'totalBorrows', 'totalSupply','utilization_ratio']], on='date')

USD_ETH['matchBorrow'] = USD_ETH['matchBorrowUSD']/USD_ETH['underlyingPriceUSD']
chart3 = alt.Chart(USD_ETH).mark_line().encode(
    x='date:T',
    y='matchBorrow:Q',
    tooltip=['date:T', 'matchBorrow:Q'],
    color=alt.value('black')
).properties(
    width=600,
    height=300
).interactive()
#(chart1 +chart2).properties(title = "Price of ETH (orange), and quantity of USDC borrowed with ETH(black)").resolve_scale(y='independent')|(chart1 +chart3).properties(title = "Price of ETH (orange), and quantity of ETH borrowed with USDC(black)").resolve_scale(y='independent')

USD_ETH['matchCollateral'] = USD_ETH['matchCollateralUSD']/USD_ETH['underlyingPriceUSD']
print(np.mean(ETH_USD['matchBorrowUSD']))
print(np.mean(ETH_USD['matchCollateralUSD']))


260007351.96617797
685196804.8699491


In [4]:
plt.figure(figsize=(8, 6))
plt.scatter(ETH_USD['borrowed_amount_change'], ETH_USD['eth_price_change'], color='blue', alpha=0.5)
plt.title('borrowed amount on eth price')
plt.xlabel('ETH price change')
plt.ylabel('borrowed amount change')
plt.show()

KeyError: 'borrowed_amount_change'

<Figure size 800x600 with 0 Axes>

**Regressing collateral on price of ETH : here I use liability df to test if the price goes up, do we use more and more ETH for borrowing USDc (that would be long position)?**

In [5]:
X = ETH_USD[['eth_price_change', 'borrowRate', 'supplyRate', 'total_supply_change', 'total_borrows_change', 'utilization_ratio', 'gas_fee_usd' ]]
correlation_matrix = X.corr()
print(correlation_matrix)
plt.figure(figsize=(8, 6))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f", linewidths=.5)
plt.title('Correlation Heatmap')
plt.show()
print(ETH_USD)

KeyError: "['eth_price_change', 'total_supply_change', 'total_borrows_change'] not in index"

In [17]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
ETH_USD['eth_price_change'] = ETH_USD['underlyingPriceUSD'].pct_change()
ETH_USD['borrowed_amount_change'] = ETH_USD['matchBorrow'].pct_change()
ETH_USD['total_supply_change'] = ETH_USD['totalSupply'].pct_change()
ETH_USD['total_borrows_change'] = ETH_USD['totalBorrows'].pct_change()

# Drop rows with NaN or infinite values
ETH_USD = ETH_USD.replace([np.inf, -np.inf], np.nan)
ETH_USD = ETH_USD.dropna()
y = ETH_USD['borrowed_amount_change']

X_init = ETH_USD[['eth_price_change', 'borrowRate', 'total_supply_change']]
#scaler = StandardScaler()
#X_init = scaler.fit_transform(X_init)

init_model = sm.OLS(y, sm.add_constant(X_init)).fit(cov_type='HC3')

print(init_model.summary())
#print(np.nan in X_init or np.nan in y)


X = ETH_USD[['eth_price_change', 'borrowRate', 'total_supply_change', 'gas_fee_usd' ]]

def forward_selection(X, y):
    selected_features = []
    remaining_features = list(X.columns)
    current_score = 0.0
    
    while remaining_features:
        best_score = 0.0
        best_feature = None
        
        for feature in remaining_features:
            model_features = selected_features + [feature]
            X_subset = X[model_features]
            X_subset = sm.add_constant(X_subset)  
            
            model = sm.OLS(y, X_subset).fit(cov_type='HC3')  
            r_squared = model.rsquared_adj
            
            if r_squared > best_score:
                best_score = r_squared
                best_feature = feature
        
        if best_score > current_score:
            selected_features.append(best_feature)
            remaining_features.remove(best_feature)
            current_score = best_score
        else:
            break
    
    return selected_features


selected_features = forward_selection(X, y)
print("Selected features with HC3:", selected_features)

# Build the final model 
final_model_features = sm.add_constant(X[selected_features])
final_model = sm.OLS(y, final_model_features).fit(cov_type='HC3')


print(final_model.summary())


                              OLS Regression Results                              
Dep. Variable:     borrowed_amount_change   R-squared:                       0.493
Model:                                OLS   Adj. R-squared:                  0.463
Method:                     Least Squares   F-statistic:                    0.9610
Date:                    Thu, 15 Feb 2024   Prob (F-statistic):              0.418
Time:                            12:46:08   Log-Likelihood:                -45.377
No. Observations:                      54   AIC:                             98.75
Df Residuals:                          50   BIC:                             106.7
Df Model:                               3                                         
Covariance Type:                      HC3                                         
                          coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------

In [ ]:
# Plot residuals
y_pred = final_model.predict(final_model_features)
residuals = y - y_pred
plt.figure(figsize=(8, 6))
plt.scatter(y_pred, residuals, color='blue', alpha=0.5)
plt.title('Residuals Plot')
plt.xlabel('Predicted Values')
plt.ylabel('Residuals')
plt.axhline(y=0, color='red', linestyle='--')  # Add a horizontal line at y=0
plt.show()

plot_data2 = pd.DataFrame({'date': USD_ETH['date'], 'Actual (y)': y, 'Predicted (y_pred)': y_pred})

print(mean_squared_error(y, y_pred))
print(max(residuals))
print(np.mean(ETH_USD['borrowed_amount_change']))
print(mean_absolute_error(y, y_pred))
print(mean_squared_error(ETH_USD['matchBorrow'], ETH_USD['totalBorrows']))

In [ ]:
plot_data = pd.DataFrame({'date': ETH_USD['date'], 'Actual (y)': y, 'Predicted (y_pred)': y_pred})
# Altair Line Plot
line_plot = alt.Chart(plot_data).mark_line().encode(
    x='date:T',
    y=alt.Y('Actual (y):Q', title='Value'),
    color=alt.value('blue'),
    tooltip=['date:T', 'Actual (y):Q']
).properties(
    width=600,
    height=400,
    title='Line Plot of Actual vs Predicted Values over Time'
)

# Add Predicted (y_pred) as an overlay
overlay = alt.Chart(plot_data).mark_line(color='red').encode(
    x='date:T',
    y='Predicted (y_pred):Q',
    tooltip=['date:T', 'Predicted (y_pred):Q']
).interactive()

(line_plot + overlay)


**Regressing USD-ETH borrow on price of ETH : here I use liability df to test if the price goes up, do we use more and more USDC for borrowing ETH (that would be short position)?**

In [ ]:
USD_ETH['eth_price_change'] = USD_ETH['underlyingPriceUSD'].pct_change()
USD_ETH['coll_change'] = USD_ETH['matchCollateral'].pct_change()
USD_ETH['total_supply_change'] = USD_ETH['totalSupply'].pct_change()
USD_ETH['total_borrows_change'] = USD_ETH['totalBorrows'].pct_change()


# Drop rows with NaN or infinite values
USD_ETH = USD_ETH.replace([np.inf, -np.inf], np.nan)
USD_ETH = USD_ETH.dropna()

#model with all covariates
y = USD_ETH['total_supply_change']
X_init = sm.add_constant(USD_ETH[['eth_price_change', 'borrowRate', 'supplyRate', 'utilization_ratio']])
init_model = sm.OLS(y, X_init).fit(cov_type='HC3')
print(init_model.summary())

print(X_init.corr())
X = USD_ETH[['eth_price_change', 'borrowRate', 'supplyRate', 'utilization_ratio']]

#forward selection
selected_features = forward_selection(X, y)
print("Selected features:", selected_features)


final_model_features = sm.add_constant(X[selected_features])
final_model2 = sm.OLS(y, final_model_features).fit(cov_type='HC3')


print(final_model2.summary())


In [ ]:
y_pred = final_model2.predict(final_model_features)
residuals = y - y_pred

# Plot residuals
plt.figure(figsize=(8, 6))
plt.scatter(y_pred, residuals, color='blue', alpha=0.5)
plt.title('Residuals Plot')
plt.xlabel('Predicted Values')
plt.ylabel('Residuals')
plt.axhline(y=0, color='red', linestyle='--')  # Add a horizontal line at y=0
plt.show()

plot_data2 = pd.DataFrame({'date': USD_ETH['date'], 'Actual (y)': y, 'Predicted (y_pred)': y_pred})

print(max(residuals**2))

In [ ]:


# Altair Line Plot
line_plot2 = alt.Chart(plot_data2).mark_line().encode(
    x='date:T',
    y=alt.Y('Actual (y):Q', title='Value'),
    color=alt.value('blue'),
    tooltip=['date:T', 'Actual (y):Q']
).properties(
    width=600,
    height=400,
    title='Line Plot of Actual vs Predicted Values over Time'
)

# Add Predicted (y_pred) as an overlay
overlay2 = alt.Chart(plot_data2).mark_line(color='red').encode(
    x='date:T',
    y='Predicted (y_pred):Q',
    tooltip=['date:T', 'Predicted (y_pred):Q']
).interactive()

(line_plot2 + overlay2)

**Now let's check the elasticity of the ETH - USD borrowing (if the price or other factors for ETH goes up, how much does the amount borrowed/ collateral vary)**

In [ ]:

# Calculate elasticity
ETH_USD['elasticity'] = ETH_USD['borrowed_amount_change'] / ETH_USD['eth_price_change']
ETH_USD['elasticity'] = ETH_USD['elasticity'].replace([np.inf, -np.inf, np.nan], 0)
print(np.mean(ETH_USD['elasticity']))


chart = alt.Chart(ETH_USD).mark_line().encode(
    x='date:T',
    y='elasticity:Q',
    tooltip=['date:T', 'elasticity:Q'],
    color=alt.value('black')
).properties(
    title='Price elasticity of Borrowing for USD with ETH',
    width=500,
    height=300
).interactive()
chart

